In [ ]:
############## PLEASE RUN THIS CELL FIRST! ###################

# import everything and define a test runner function
from importlib import reload
from helper import run
import ecc

# Schnorr Verification
* $e$ is the private key, $eG=P$, $z$ is the message, $k$ is the nonce, $kG=R$
* $\mathcal{H}$ is a tagged hash (BIP0340/challenge)
* $d$ challenge, $d=\mathcal{H}(R||P||z)$
* Signature is $(R,s)$ where $s=k + e d$
* Verify $R=sG-dP$
* $$sG-dP =(k+e d)G-dP =kG+d(eG)-dP=R+dP-dP=R$$


In [ ]:
from ecc import S256Point, SchnorrSignature, G, N
from hash import hash_challenge
from helper import big_endian_to_int
msg = b"I'm learning Schnorr Signatures!"
sig_raw = bytes.fromhex("3b5b656f623e314fcff97b44f93d4452992856e65fe0268a77a9a94c626eb1b11e8bcea138a15c185633fd66a7c1683843daa332c9d9e27a7178389d338521ab")
sig = SchnorrSignature.parse(sig_raw)
xonly = bytes.fromhex("a8a28557947025fe0646660677c09a757a3bce148d99fac9368439a13df6ea1a")
p = S256Point.parse(xonly)
commitment = sig.r.xonly() + p.xonly() + msg
d = big_endian_to_int(hash_challenge(commitment)) % N
target = sig.s * G - d * p
print(target == sig.r)

### Exercise 1

Verify this Schnorr Signature

Pubkey = cbaa648dbfe734646ce958e2f14a874149fae4010fdeabde4bae6a732537fd91
Signature = 2ae68f873376a0ff302258964632f7b98b21e3bbc72dcc8fb31de8acf01696b951f3dbb6fc5532558219472fb63e061f9a4c7d1760cc588da551c74374cd0de4
Message = 1a84547db188f0b1d2c9f0beac230afebbd5e6e6c1a46fc69841815194bf8612



In [ ]:
# Exercise 1

from ecc import SchnorrSignature, S256Point, N, G
from hash import hash_challenge
from helper import big_endian_to_int
p_raw = bytes.fromhex("cbaa648dbfe734646ce958e2f14a874149fae4010fdeabde4bae6a732537fd91")
p = S256Point.parse(p_raw)
sig_raw = bytes.fromhex("2ae68f873376a0ff302258964632f7b98b21e3bbc72dcc8fb31de8acf01696b951f3dbb6fc5532558219472fb63e061f9a4c7d1760cc588da551c74374cd0de4")
sig = SchnorrSignature.parse(sig_raw)
msg = bytes.fromhex("1a84547db188f0b1d2c9f0beac230afebbd5e6e6c1a46fc69841815194bf8612")
# create the commitment: R || P || z (points should be xonly)

# d is the hash_challenge of the commitment as a big endian int

# check that R=sG-dP


### Exercise 2




#### Make [this test](/edit/session1/ecc.py) pass: `ecc.py:SchnorrTest:test_verify`

In [ ]:
# Exercise 2

reload(ecc)
run(ecc.SchnorrTest('test_verify'))

# Schnorr Signing
* $eG=P$, $m$ message, $k$ random
* $kG=R$, $H$ is <code>hash_challenge</code.
* $s=k+e H(R||P||m)$ where $R$ and $P$ are $x$-only
* Signature is $(R,s)$


In [ ]:
from ecc import PrivateKey, N, G
from hash import hash_challenge
from helper import big_endian_to_int
priv = PrivateKey(12345)
e = priv.even_secret()
msg = b"I'm learning Schnorr Signatures!"
k = 21016020145315867006318399104346325815084469783631925097217883979013588851039
r = k * G
if not r.even:
    k = N - k
    r = k * G
challenge = r.xonly() + priv.point.xonly() + msg
d = big_endian_to_int(hash_challenge(challenge)) % N
s = (k + e * d) % N
sig = SchnorrSignature(r, s)
if not priv.point.verify_schnorr(msg, sig):
    raise RuntimeError("Bad Signature")
print(sig.serialize().hex())

### Exercise 3

Sign the message b"Schnorr Signatures adopt Taproot" with the private key 21,000,000



In [ ]:
# Exercise 3

from ecc import PrivateKey, N, G
from hash import hash_challenge
from helper import big_endian_to_int
priv = PrivateKey(21000000)
msg = b"Schnorr Signatures adopt Taproot"
k = 987654321
# get e using the even_secret method on the private key
e = priv.even_secret() #/
# calculate the nonce point R which is kG

# if R's not even, negate the k and recalculate R

    # set k to N - k

    # recalculate R

# calculate the commitment which is: R || P || msg

# d is the hash_challenge of the commitment as a big endian integer mod N

# calculate s = (k+ed) mod N

# create a SchnorrSignature object using the R and s

# check that this schnorr signature verifies


# print the serialized hex of the signature


### Exercise 4




#### Make [this test](/edit/session1/ecc.py) pass: `ecc.py:SchnorrTest:test_sign`

In [ ]:
# Exercise 4

reload(ecc)
run(ecc.SchnorrTest('test_sign'))

# Nonce ($k$) Creation
* Start with a random number $a$, which is then hashed
* Xor the result $\mathcal{H_1}(a)$ with the secret $e$
* Then hash with the message $z$ to generate the $k$
* $P=eG$, $\mathcal{H_1}$ is <code>hash_aux</code>, $\mathcal{H_2}$ is <code>hash_nonce</code>
* $x = \mathcal{H_1}(a) \oplus e$, $k=\mathcal{H_2}(x||P||z)$
* $k$ is unique to both the secret and the message
* 32 0-bytes $a$ can be used to create a deterministic $k$


In [ ]:
# example of nonce creation
from ecc import PrivateKey, N
from hash import sha256, hash_aux, hash_nonce
from helper import big_endian_to_int, int_to_big_endian, xor_bytes
aux = bytes([0] * 32)
private_key = PrivateKey(21000000)
p = private_key.point
e = private_key.even_secret()
msg = sha256(b"Nonce generation is spectacular!")
x = xor_bytes(int_to_big_endian(e, 32), hash_aux(aux))
k = big_endian_to_int(hash_nonce(x + p.xonly() + msg)) % N
print(hex(k))

### Exercise 5

Sign the message b"Secure Deterministic Nonce made!" with the private key 837,120,557



In [ ]:
# Exercise 5

from ecc import PrivateKey, N, G
from hash import sha256, hash_aux, hash_nonce
from helper import big_endian_to_int, int_to_big_endian, xor_bytes
priv = PrivateKey(21000000)
point = priv.point
msg = b"Secure Deterministic Nonce made!"
# get e using the even_secret method on the private key
e = priv.even_secret() #/
# use the 32-bytes of 0's for the auxillary
a = bytes([0] * 32) #/
# x=e⊕H(a) where ⊕ is xor_bytes, H is hash_aux and a is the auxillary
x = xor_bytes(int_to_big_endian(e, 32), hash_aux(a)) #/
# k=H(x||P||z) where H is hash_nonce, P is the xonly of the point and z is the message
k = big_endian_to_int(hash_nonce(x + point.xonly() + msg)) % N #/
# calculate R which is kG

# if R is not even negate the k

    # set k to N - k

    # recalculate R

# calculate the commitment which is: R || P || msg

# d=H(commitment) where H is the hash_challenge

# s=(k+ed) mod N

# create a SchnorrSignature object using the R and s

# check that this schnorr signature verifies


# print the serialized hex of the signature


### Exercise 6




#### Make [this test](/edit/session1/ecc.py) pass: `ecc.py:SchnorrTest:test_bip340_k`

In [ ]:
# Exercise 6

reload(ecc)
run(ecc.SchnorrTest('test_bip340_k'))

# Batch Verification
* Pubkeys are $P_i$, Signatures are $(R_i,s_i)$
* Challenges are $d_i=\mathcal{H}(R_i||P_i||z_i)$
* $R_i=s_iG-d_iP_i$
* $\sum{R_i}=\sum{s_iG}-\sum{d_iP_i}$
* $(\sum{s_i})G=\sum{R_i}+\sum{d_iP_i}$
* Fewer total operations!


### Exercise 7

Batch Verify these two Schnorr Signatures

Pubkey 1 = cbaa648dbfe734646ce958e2f14a874149fae4010fdeabde4bae6a732537fd91
Pubkey 2 = e79c4eb45764bd015542f6779cc70fef44b7a2432f839264768288efab886291

Signature 1 = 2ae68f873376a0ff302258964632f7b98b21e3bbc72dcc8fb31de8acf01696b951f3dbb6fc5532558219472fb63e061f9a4c7d1760cc588da551c74374cd0de4
Signature 2 = b6e52f38bc24f1420c4fdae8fa0f04b9b0374a12f18fd4699b06df53eb1386bfa88c1835cd19470cf8c76550eb549c988f9c8fac00cc56fadd4fcc3bf9d8800e

Message 1 = 1a84547db188f0b1d2c9f0beac230afebbd5e6e6c1a46fc69841815194bf8612
Message 2 = af1c325abcb0cced3a4166ce67be1db659ae1dd574fe49b0f2941d8d4882d62c



In [ ]:
# Exercise 7

from ecc import SchnorrSignature, S256Point, N, G
from hash import hash_challenge
from helper import big_endian_to_int
p1_raw = bytes.fromhex("cbaa648dbfe734646ce958e2f14a874149fae4010fdeabde4bae6a732537fd91")
p2_raw = bytes.fromhex("e79c4eb45764bd015542f6779cc70fef44b7a2432f839264768288efab886291")
p1 = S256Point.parse(p1_raw)
p2 = S256Point.parse(p2_raw)
sig1_raw = bytes.fromhex("2ae68f873376a0ff302258964632f7b98b21e3bbc72dcc8fb31de8acf01696b951f3dbb6fc5532558219472fb63e061f9a4c7d1760cc588da551c74374cd0de4")
sig2_raw = bytes.fromhex("b6e52f38bc24f1420c4fdae8fa0f04b9b0374a12f18fd4699b06df53eb1386bfa88c1835cd19470cf8c76550eb549c988f9c8fac00cc56fadd4fcc3bf9d8800e")
sig1 = SchnorrSignature.parse(sig1_raw)
sig2 = SchnorrSignature.parse(sig2_raw)
msg1 = bytes.fromhex("1a84547db188f0b1d2c9f0beac230afebbd5e6e6c1a46fc69841815194bf8612")
msg2 = bytes.fromhex("af1c325abcb0cced3a4166ce67be1db659ae1dd574fe49b0f2941d8d4882d62c")
# define s as the s_i sum (make sure to mod by N)

# define r as the signatures' r sum

# create the commitments: R_i||P_i||m_i


# d_i are the challenges which are hash_challenge of the commitments as big endian ints mod N


# d is the sum of the d_i P_i's

# check that sG=R+d


# Spending from the KeyPath
* $m$ is the Merkle Root of the ScriptPath
* Tweak $t$ and $P$ create $Q$, the external pubkey
* $t=\mathcal{H}(P||m)$ where $\mathcal{H}$ is <code>hash_taptweak</code>
* $Q=P+tG$, and $eG=P$ which means $Q=eG+tG$ and $Q=(e+t)G$
* $e+t$ is your private key, whose public key is $Q$
* Witness has a single element, the Schnorr Signature
* If you don't want a script path, $m$ is the empty string


In [ ]:
from ecc import S256Point, G
from hash import hash_taptweak
from helper import big_endian_to_int
from script import P2TRScriptPubKey
internal_pubkey_raw = bytes.fromhex("cbaa648dbfe734646ce958e2f14a874149fae4010fdeabde4bae6a732537fd91")
p = S256Point.parse(internal_pubkey_raw)
m = b""
t = big_endian_to_int(hash_taptweak(p.xonly() + m))
q = p + t * G
script_pubkey = P2TRScriptPubKey(q)
print(script_pubkey)

### Exercise 8

Make a P2TR ScriptPubKey with no Script Path using the private key 9284736473



In [ ]:
# Exercise 8

from ecc import PrivateKey, G
from hash import hash_taptweak
from helper import big_endian_to_int
from script import P2TRScriptPubKey
priv = PrivateKey(9284736473)
# get the internal pubkey, P

# set the merkle root to the empty stning, m

# t is the hash_taptweak of the internal pubkey xonly and the merkle root as a big endian integer

# Q = P + tG

# use P2TRScriptPubKey to create the ScriptPubKey

# print the ScriptPubKey


### Exercise 9




#### Make [this test](/edit/session1/ecc.py) pass: `ecc.py:TapRootTest:test_default_tweak`

In [ ]:
# Exercise 9

reload(ecc)
run(ecc.TapRootTest('test_default_tweak'))

### Exercise 10




#### Make [this test](/edit/session1/ecc.py) pass: `ecc.py:TapRootTest:test_tweaked_key`

In [ ]:
# Exercise 10

reload(ecc)
run(ecc.TapRootTest('test_tweaked_key'))

### Exercise 11




#### Make [this test](/edit/session1/ecc.py) pass: `ecc.py:TapRootTest:test_private_tweaked_key`

In [ ]:
# Exercise 11

reload(ecc)
run(ecc.TapRootTest('test_private_tweaked_key'))

### Exercise 12




#### Make [this test](/edit/session1/ecc.py) pass: `ecc.py:TapRootTest:test_p2tr_script`

In [ ]:
# Exercise 12

reload(ecc)
run(ecc.TapRootTest('test_p2tr_script'))

# P2TR Addresses
* Segwit v0 uses Bech32
* Taproot (Segwit v1) uses Bech32m
* Bech32m is different than Bech32 (BIP350)
* Has error correcting capability and uses 32 letters/numbers
* Segwit v0 addresses start with <code>bc1q</code> and p2wpkh is shorter than p2wsh
* Segwit v1 addresses start with <code>bc1p</code> and they're all one length


In [ ]:
# Example of getting a p2tr address
from ecc import S256Point
internal_pubkey_raw = bytes.fromhex("cbaa648dbfe734646ce958e2f14a874149fae4010fdeabde4bae6a732537fd91")
p = S256Point.parse(internal_pubkey_raw)
print(p.p2tr_address())
print(p.p2tr_address(network="signet"))

### Exercise 13

Make your own Signet P2TR Address

Submit your address at [this link]()



In [ ]:
# Exercise 13

from ecc import PrivateKey
from hash import sha256
from helper import big_endian_to_int
my_email = b"<fill this in with your email>"
my_secret = big_endian_to_int(sha256(my_email))
# create the private key object

# get the public point

# print the p2tr_address with network set to "signet"
